In [14]:
from tensorboard.backend.event_processing import event_accumulator
import os
import numpy as np
import pandas as pd
import json

def load_metric_from_event(event_name): 
    event_path = os.path.join("runs", event_name)
    event_name = os.listdir(event_path)[0]
    event_path = os.path.join(event_path, event_name)

    # only read scalars
    ea = event_accumulator.EventAccumulator(event_path, size_guidance={event_accumulator.SCALARS: 0})

    # load scalars
    ea.Reload()

    return get_best_epoch_metrics(ea)

def get_best_epoch_metrics(ea):
    test_accs = ea.Scalars(f"test/acc")
    test_acc_vals = [acc.value for acc in test_accs]
    test_best_acc_idx = np.argmax(test_acc_vals)
    test_best_acc = np.max(test_acc_vals)
    test_best_epoch = test_accs[test_best_acc_idx].step

    train_accs = ea.Scalars(f"train/acc")
    train_best_acc = train_accs[test_best_acc_idx].value
    try:
        train_img_clas_accs = ea.Scalars(f"train/img_class_acc")
        train_best_img_clas_acc = train_img_clas_accs[test_best_acc_idx].value

        
        test_img_clas_accs = ea.Scalars(f"train/img_class_acc")
        test_best_img_clas_acc = test_img_clas_accs[test_best_acc_idx].value
    except KeyError:
        train_best_img_clas_acc = np.nan
        test_best_img_clas_acc = np.nan
    
    return [test_best_epoch, train_best_acc, test_best_acc, train_best_img_clas_acc, test_best_img_clas_acc]

## Define important variables

In [15]:
# events in tensorboard
events = ['24_03_2021_10_33_16_task_standard_seed_7', '24_03_2021_10_37_04_task_standard_seed_122', '24_03_2021_10_39_50_task_standard_seed_809', '24_03_2021_10_43_50_task_standard_seed_7', '24_03_2021_11_00_48_task_standard_seed_122', '24_03_2021_11_18_26_task_standard_seed_809', '24_03_2021_11_35_57_task_standard_seed_7', '24_03_2021_11_54_58_task_standard_seed_122', '24_03_2021_12_13_03_task_standard_seed_809', '24_03_2021_12_30_12_task_img_clas_seed_7', '24_03_2021_12_37_57_task_img_clas_seed_122', '24_03_2021_12_44_03_task_img_clas_seed_809', '24_03_2021_13_05_47_task_img_clas_seed_7', '24_03_2021_13_27_19_task_img_clas_seed_122', '24_03_2021_13_48_31_task_img_clas_seed_809', '24_03_2021_13_59_50_task_img_clas_seed_7', '24_03_2021_14_21_20_task_img_clas_seed_122', '24_03_2021_14_43_14_task_img_clas_seed_809', '24_03_2021_15_05_02_task_target_clas_seed_7', '24_03_2021_15_09_49_task_target_clas_seed_122', '24_03_2021_15_14_35_task_target_clas_seed_809', '24_03_2021_15_19_17_task_target_clas_seed_7', '24_03_2021_15_41_05_task_target_clas_seed_122', '24_03_2021_16_02_59_task_target_clas_seed_809', '24_03_2021_16_24_57_task_target_clas_seed_7', '24_03_2021_16_47_18_task_target_clas_seed_122', '24_03_2021_17_09_55_task_target_clas_seed_809']

# choose which columsn to include
include_cols = ["task", "ic_loss_weight", "num_imgs", "same_class_prob", "seed"]

In [132]:
results = []
args_dir = os.path.join("args")
for event_name in events:
    # get parameters
    with open(f"{args_dir}\\args_{event_name}.json") as json_file:
        params = json.load(json_file)
    # wrap each value into a list
    params = {key:[value] for key, value in params.items()}

    # create dataframe and add parameters
    results_df = pd.DataFrame(params)[include_cols]

    # add metrics
    metric_cols = ["best epoch", "best train acc", "best test acc", "best train img acc", "best test img acc"]
    results_df.loc[:, metric_cols] = load_metric_from_event(event_name)
    # append to final table
    results.append(results_df)

results = pd.concat(results)
results["success"] = (results["best test acc"]>0.97).astype(int)

In [133]:
results

,task,ic_loss_weight,num_imgs,same_class_prob,seed,best epoch,best train acc,best test acc,best train img acc,best test img acc,success
0,standard,1.000,2,0.000,7,3,0.968,0.976,nan,nan,1
0,standard,1.000,2,0.000,122,2,0.951,0.971,nan,nan,1
0,standard,1.000,2,0.000,809,3,0.967,0.975,nan,nan,1
0,standard,1.000,2,0.500,7,7,0.946,0.961,nan,nan,0
0,standard,1.000,2,0.500,122,7,0.918,0.936,nan,nan,0
0,standard,1.000,2,0.500,809,6,0.927,0.944,nan,nan,0
0,standard,1.000,2,1.000,7,13,0.907,0.941,nan,nan,0
0,standard,1.000,2,1.000,122,10,0.914,0.936,nan,nan,0
0,standard,1.000,2,1.000,809,9,0.928,0.946,nan,nan,0
0,img_clas,1.000,2,0.000,7,5,0.961,0.971,0.918,0.918,1


In [134]:
# success_results = results[results["success"]==1].groupby(by=["task", "same_class_prob"])["best epoch"].mean()

In [136]:
results = results.drop(["ic_loss_weight", "num_imgs", "best epoch", "best train acc", "best train img acc"], axis=1)
results = results.groupby(by=["task", "same_class_prob"]).mean().drop(["seed"], axis=1) #.agg({'best test acc' : ['mean'], 'best test img acc':['mean'], "success":['mean']}) 

In [137]:
results = results.reindex(index = ['standard','img_clas','target_clas'], level="task").fillna('-')

In [138]:
pd.options.display.float_format = '{:,.3f}'.format

In [139]:
results = results.drop(["success"], axis=1)

In [140]:
results

best test acc best test img acc
task        same_class_prob                                 
standard    0.000                    0.974                 -
            0.500                    0.947                 -
            1.000                    0.941                 -
img_clas    0.000                    0.964             0.909
            0.500                    0.958             0.887
            1.000                    0.937             1.000
target_clas 0.000                    0.975             0.339
            0.500                    0.919             0.434
            1.000                    0.917             0.505

In [123]:
print(results.to_latex())

\begin{tabular}{llrrl}
\toprule
            &       &  best epoch &  best test acc & best test img acc \\
task & same\_class\_prob &             &                &                   \\
\midrule
standard & 0.000 &       2.667 &          0.974 &                 - \\
            & 0.500 &       6.667 &          0.947 &                 - \\
            & 1.000 &      10.667 &          0.941 &                 - \\
img\_clas & 0.000 &       5.000 &          0.964 &             0.909 \\
            & 0.500 &       8.667 &          0.958 &             0.887 \\
            & 1.000 &       9.667 &          0.937 &             1.000 \\
target\_clas & 0.000 &       3.000 &          0.975 &             0.339 \\
            & 0.500 &       6.667 &          0.919 &             0.434 \\
            & 1.000 &       8.667 &          0.917 &             0.505 \\
\bottomrule
\end{tabular}

